In [6]:
# load data files
# PLEASE MODIFY THE PATH OF DATA!!!!!!!!!!!!!!!!!!!!!!!!!
# ===========================================================================================
# 下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下

# data_subfold = "data_2023_03_01-16_48_29"    # the data from Lukas
# data_subfold = "rosbag2_2023_06_05-16_57_44"
data_subfold = "rosbag2_2023_06_05-17_22_10"
# data_subfold = "rosbag2_2023_06_05-17_26_41"

save_merged_data = False

execfile("data_process_from_rosbags.py")
# 上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上
# ===========================================================================================


# Files
from model import *
from model_kf import *
from model_all import *

# Packages
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

Defined Variables:
    df_command
    df_imu
    df_velocity
    df_tf
    df_merge    # merged from 'df_command', 'df_velocity', 'df_imu' and 'df_tf'

Finish!


Loading BokehJS ...

In [9]:
df_imu

,__time,imu.header.stamp,imu.w_x,imu.w_y,imu.w_z,imu.a_x,imu.a_y,imu.a_z
0,1.685979e+09,1.685979e+09,-0.006981,-0.001745,-0.001745,0.068647,-9.728196,0.568786
1,1.685979e+09,1.685979e+09,-0.008727,-0.003491,-0.001745,0.068647,-9.728196,0.568786
2,1.685979e+09,1.685979e+09,-0.006981,0.000000,0.000000,0.068647,-9.728196,0.578592
3,1.685979e+09,1.685979e+09,-0.001745,-0.006981,0.000000,0.068647,-9.728196,0.578592
4,1.685979e+09,1.685979e+09,-0.001745,-0.001745,0.000000,0.058840,-9.728196,0.588399
...,...,...,...,...,...,...,...,...
3749,1.685979e+09,1.685979e+09,-0.006981,-0.003491,0.000000,0.098066,-9.738003,0.490332
3750,1.685979e+09,1.685979e+09,-0.003491,0.000000,0.000000,0.098066,-9.738003,0.490332
3751,1.685979e+09,1.685979e+09,-0.003491,-0.001745,0.000000,0.098066,-9.738003,0.490332
3752,1.685979e+09,1.685979e+09,-0.005236,0.000000,0.000000,0.088260,-9.738003,0.490332


# KF with velocity data

In [ ]:
imu = df_imu.iloc[:,[1, -1, 3]].rename(columns={'imu.header.stamp': 'time'})
velocity = df_velocity.rename(columns={'__time': 'time'})

df_data = imu.merge(velocity, how='outer', on='time')
df_data.sort_values(by=['time'], inplace=True)
df_data.reset_index(inplace=True, drop=True)
data = df_data.to_numpy()
df_data

In [ ]:
# ============================================ Simulation =============================================
# needed data for simulation
u_k = np.append(data[:, 1].reshape(1, -1), data[:, 2].reshape(1, -1), axis=0)  # a_z, w_y, with nan, shape: (2, n)
y_k = - data[:, 3] / 1000  # unit of data is mm/s and data is inverse
switch = np.isnan(data[:, 3]) # switch, to determine prediction or correction in KF, if False,then correction
Ts = np.diff(data[:, 0])

x0 = [2.668, -3.346, 0, 0]
Q = np.diag([1, 1, 1, 1])
R = np.array([[0.1]])
P0 = np.diag([10, 10, 10, 10])

x_all = sim_kf(x0, u_k, y_k, switch, Ts, Q, R, P0)
print("shape of result", x_all.shape)

# ============================================ Plot =============================================
p_total = []

# simulation curve
p = plot_config(width=800, height=800, title='curve', x_label='x [ unknown ]', y_label='y [ unknown ]')
p.match_aspect=True
p.aspect_scale=1
p.triangle(x_all[0][0], x_all[1][0], legend_label='start_sim', fill_color="black", line_color='black', size=20)
p.circle(x=x_all[0], y=x_all[1], legend_label='sim_curve', fill_color="royalblue", line_color='royalblue', size=5)  # [0:3000]
p.asterisk(x_all[0][-1], x_all[1][-1], line_color="black", legend_label='end_sim', size=20)

# tf curve
p.triangle(Dtf["tf.x"].to_numpy()[0], Dtf["tf.y"].to_numpy()[0], legend_label='start_tf', fill_color="red", line_color='red', size=20)
p.circle(x=Dtf["tf.x"], y=Dtf["tf.y"], legend_label='tf_curve', line_color='orange', fill_color='orange', size=3)
p.asterisk(Dtf["tf.x"].to_numpy()[-1], Dtf["tf.y"].to_numpy()[-1], line_color="red", legend_label='end_tf', size=20)
p_total.append(p)

# simulation states
t0 = df_command["__time"][0]  # regard the global time stamp of first data point in "command" as 0s

p2 = plot_config(width=900, height=600, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p2.line(x=data[:, 0] - t0, y=x_all[0], legend_label='x', line_width=2, line_color='green')
p2.line(x=data[:, 0] - t0, y=x_all[1], legend_label='y', line_width=2, line_color='black')
p2.line(x=data[:, 0] - t0, y=x_all[2], legend_label='theta', line_width=2, line_color='royalblue')
p2.line(x=data[:, 0] - t0, y=x_all[3], legend_label='v', line_width=2, line_color='crimson')
p_total.append(p2)


show(p, notebook_handle=True)
show(p2, notebook_handle=True)
# or
# from bokeh.io import output_file
# output_file(filename="./simulation_kf_header_stamp.html", title="KF_imu_velocity vs tf header_stamp")
# show(gridplot(p_total, ncols=2))

# KF with velocity data and tf data

In [7]:
Dimu = df_imu.iloc[:,[1, -1, 3]].rename(columns={'imu.header.stamp': 'time'}) # or
# Dimu = df_imu.iloc[:,[0, -1, 3]].rename(columns={'__time': 'time'}) # global time

Dvelocity = df_velocity.rename(columns={'__time': 'time'}) # global time

# Dtf = df_tf.iloc[:,[0, 2, 3, -1]].rename(columns={'__time': 'time'}) # global time
# or
d_time = df_tf.iloc[:,1][0] - df_tf.iloc[:,0][0] # time difference between header.stamp and __time of TF by the first data point
Dtf = df_tf.iloc[:,[1, 2, 3, -1]].rename(columns={'tf.header.stamp': 'time'})# use header.stamp
Dtf["time"] = Dtf["time"] - d_time


# # compensation of tf delay
# tf_delay = 0.84
# Dtf["time"] = Dtf["time"] - tf_delay

df_data = Dimu.merge(Dvelocity, how='outer', on='time')
df_data = df_data.merge(Dtf, how='outer', on='time')
df_data.sort_values(by=['time'], inplace=True)
df_data.reset_index(inplace=True, drop=True)
data = df_data.to_numpy()
# df_data.head(10)

In [8]:
# ==========================================Simulation================================================
yaw_z = data[:,-1].reshape(1, -1)    # the yaw angle of tf, shape: (1, n)

u_k = np.append(data[:,1].reshape(1, -1), data[:, 2].reshape(1, -1), axis=0)  # a_z, w_y, with nan, shape: (2, n)
y_k_vel = - data[:, 3] / 1000  # unit of data is mm/s and data is inverse
y_k_tf = data[:, 4:7].T    # [x, y, yaw_angle].T , shape: (3, n)
Ts = np.diff(data[:, 0])    # get time steps
# Switch, if the data are NaN, True. To determine prediction or correction in KF
s_imu = np.isnan(data[:, 1])
s_vel = np.isnan(data[:, 3])
s_tf = np.isnan(data[:, 4])


# Initial
x0 = [Dtf["tf.x"][0], Dtf["tf.y"][0], Dtf["tf.yaw_z"][0], 0]
Q = np.diag([1, 1, 1, 1])
R_v = np.array([[0.1]])
R_t = np.diag([0.1, 0.1, 0.1])
P0 = np.diag([10, 10, 10, 10])

x_all = sim_all(x0, u_k, Ts, y_k_vel, y_k_tf, s_imu, s_vel, s_tf, Q, R_v, R_t, P0)
print("shape of result", x_all.shape)

# ============================================ Plot =============================================
p_total = []

# simulation curve
p = plot_config(width=800, height=800, title='curve', x_label='x [ unknown ]', y_label='y [ unknown ]')
p.match_aspect=True
p.aspect_scale=1
p.triangle(x_all[0][0], x_all[1][0], legend_label='start_sim', fill_color="black", line_color='black', size=20)
p.circle(x=x_all[0], y=x_all[1], legend_label='sim_curve', fill_color="royalblue", line_color='royalblue', size=5)  # [0:3000]
p.asterisk(x_all[0][-1], x_all[1][-1], line_color="black", legend_label='end_sim', size=20)

# tf curve
p.triangle(Dtf["tf.x"].to_numpy()[0], Dtf["tf.y"].to_numpy()[0], legend_label='start_tf', fill_color="red", line_color='red', size=20)
p.circle(x=Dtf["tf.x"], y=Dtf["tf.y"], legend_label='tf_curve', line_color='orange', fill_color='orange', size=3)
p.asterisk(Dtf["tf.x"].to_numpy()[-1], Dtf["tf.y"].to_numpy()[-1], line_color="red", legend_label='end_tf', size=20)
p_total.append(p)

# simulation states
t0 = df_command["__time"][0]  # regard the global time stamp of first data point in "command" as 0s

p2 = plot_config(width=900, height=600, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p2.line(x=data[:, 0] - t0, y=x_all[0], legend_label='x', line_width=2, line_color='green')
p2.line(x=data[:, 0] - t0, y=x_all[1], legend_label='y', line_width=2, line_color='black')
p2.line(x=data[:, 0] - t0, y=x_all[2], legend_label='theta', line_width=2, line_color='royalblue')
p2.line(x=data[:, 0] - t0, y=x_all[3], legend_label='v', line_width=2, line_color='crimson')
p_total.append(p2)


show(p, notebook_handle=True)
show(p2, notebook_handle=True)
# or
# from bokeh.io import output_file
# output_file(filename=f"./simulation_all global_time.html", title="KF_all vs tf")
# output_file(filename=f"./simulation_all imu_header tf_global.html", title="KF_all vs tf imu_header tf_global")
# output_file(filename=f"./simulation_all imu_header tf_global-{tf_delay}.html", title="KF_all vs tf header_stamp tf-{tf_delay}")
# output_file(filename=f"./simulation_all imu_header tf_header.html", title="KF_all vs tf imu_header tf_header")
# show(gridplot(p_total, ncols=2))

shape of result (4, 6795)
